In [1]:
# dependencies
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import numpy as np
import pandas as pd
# import addfips
import urllib
import random
plotly.tools.set_credentials_file(username='michaelgarza', api_key='Dqqcuh2voaSqHOqQFP5W')

In [2]:
state_abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
        'District of Columbia':'District of Columbia'
}

In [3]:
# reading in csv file that contains county information to create the choropleth
# ('https://aqs.epa.gov/aqsweb/airdata/annual_aqi_by_county_2018.zip')

df_aqi = pd.read_csv("raw_data/annual_aqi_by_county_2017.csv")
df_aqi['County'] = df_aqi['County'].apply(lambda x: x.lower())
# df_aqi[df_aqi['State']==''.startswith('D')]
# df_aqi['State'].unique()
print(len(df_aqi))
df_aqi.head()

1061


,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
0,Alabama,baldwin,2017,270,241,28,1,0,0,0,108,51,36,0,0,206,0,64,0
1,Alabama,clay,2017,118,104,14,0,0,0,0,66,52,30,0,0,0,0,118,0
2,Alabama,colbert,2017,283,265,18,0,0,0,0,63,48,37,0,0,218,0,65,0
3,Alabama,dekalb,2017,359,329,30,0,0,0,0,80,50,39,0,0,315,0,44,0
4,Alabama,elmore,2017,226,221,5,0,0,0,0,58,45,35,0,0,226,0,0,0


In [4]:
df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')
# df_sample=pd.concat([df_sample, df_aqi],axis=1)
df_sample['State FIPS Code'] = df_sample['State FIPS Code'].apply(lambda x: str(x).zfill(2))
df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
df_sample['FIPS'] = df_sample['State FIPS Code'] + df_sample['County FIPS Code']

# df_sample['County'], df_sample['State'] = df_sample[df_sample['County Name/State Abbreviation'][:-4]]
#  = df_sample[['County Name/State Abbreviation']][-2:]
df_sample["County"] = df_sample['County Name/State Abbreviation'].apply(lambda x: x.split(",")[0].replace(" County", "").lower())
df_sample["State Abbreviation"] = df_sample['County Name/State Abbreviation'].apply(lambda x: x.split(", ")[-1])
df_sample["State"] = df_sample["State Abbreviation"].apply(lambda x: state_abbrev[x])
# df_sample[df_sample["State Abbreviation"]=="District of Columbia"]
print(len(df_sample))
df_sample.head()

3219


,LAUS Code,State FIPS Code,County FIPS Code,County Name/State Abbreviation,Year,Labor Force,Employed,Unemployed,Unemployment Rate (%),FIPS,County,State Abbreviation,State
0,CN0100100000000,01,001,"Autauga County, AL",2016,"25,649","24,297","1,352",5.3,01001,autauga,AL,Alabama
1,CN0100300000000,01,003,"Baldwin County, AL",2016,"89,931","85,061","4,870",5.4,01003,baldwin,AL,Alabama
2,CN0100500000000,01,005,"Barbour County, AL",2016,"8,302","7,584",718,8.6,01005,barbour,AL,Alabama
3,CN0100700000000,01,007,"Bibb County, AL",2016,"8,573","8,004",569,6.6,01007,bibb,AL,Alabama
4,CN0100900000000,01,009,"Blount County, AL",2016,"24,525","23,171","1,354",5.5,01009,blount,AL,Alabama


In [45]:
df_countydata = df_sample[['State', 'County', 'FIPS', 'Year', 'State FIPS Code', 'State Abbreviation']]
df_countydata.head()

,State,County,FIPS,Year,State FIPS Code,State Abbreviation
0,Alabama,autauga,01001,2016,01,AL
1,Alabama,baldwin,01003,2016,01,AL
2,Alabama,barbour,01005,2016,01,AL
3,Alabama,bibb,01007,2016,01,AL
4,Alabama,blount,01009,2016,01,AL


In [46]:
df_countydata = df_countydata.merge(df_aqi, how='inner', on=['State', 'County'])
# help(pd.merge)
print(len(df_countydata))
df_countydata

1002


,State,County,FIPS,Year_x,State FIPS Code,State Abbreviation,Year_y,Days with AQI,Good Days,Moderate Days,...,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
0,Alabama,baldwin,01003,2016,01,AL,2017,270,241,28,...,0,108,51,36,0,0,206,0,64,0
1,Alabama,clay,01027,2016,01,AL,2017,118,104,14,...,0,66,52,30,0,0,0,0,118,0
2,Alabama,colbert,01033,2016,01,AL,2017,283,265,18,...,0,63,48,37,0,0,218,0,65,0
3,Alabama,dekalb,01049,2016,01,AL,2017,359,329,30,...,0,80,50,39,0,0,315,0,44,0
4,Alabama,elmore,01051,2016,01,AL,2017,226,221,5,...,0,58,45,35,0,0,226,0,0,0
5,Alabama,etowah,01055,2016,01,AL,2017,360,233,125,...,0,163,62,45,0,0,133,0,227,0
6,Alabama,houston,01069,2016,01,AL,2017,281,260,20,...,0,131,48,35,0,0,203,0,78,0
7,Alabama,jefferson,01073,2016,01,AL,2017,365,144,212,...,0,129,76,55,0,2,85,70,207,1
8,Alabama,lawrence,01079,2016,01,AL,2017,51,46,5,...,0,80,48,21,0,0,0,0,51,0
9,Alabama,madison,01089,2016,01,AL,2017,336,302,34,...,0,100,51,39,0,0,230,0,52,54


In [54]:
state_aqi = pd.DataFrame(df_countydata.groupby(['FIPS','State FIPS Code','State','State Abbreviation'])['Median AQI'].mean()).reset_index()
state_aqi.head() #['Median AQI']
# help(pd.DataFrame.groupby)
# fips = df_countydata['FIPS'].tolist()
# values = df_countydata['Median AQI'].tolist()

,FIPS,State FIPS Code,State,State Abbreviation,Median AQI
0,01003,01,Alabama,AL,36
1,01027,01,Alabama,AL,30
2,01033,01,Alabama,AL,37
3,01049,01,Alabama,AL,39
4,01051,01,Alabama,AL,35


In [11]:
# df_sample['State FIPS Code'] = df_sample['State FIPS Code'].apply(lambda x: str(x).zfill(2))
# df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
# df_sample['FIPS'] = df_sample['State FIPS Code'] + df_sample['County FIPS Code']

colorscale = ["#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce"]
colors = {"good": "#1fca23", 
          "moderate": "#fffd38", 
          "usg":"#fc6621", 
          "unhealthy": "#fc0d1b", 
          "very unhealthy":"#97084c", 
          "hazardous":"#7d0425", 
          "idk":"#000000"
         }
# endpts = list(np.linspace(0, 500, len(colors.values()) - 1))
endpts = [50, 100, 150, 200, 300, 500]
# endpts = list(np.linspace(0, 500, 50))
fips = df_countydata['FIPS'].tolist()
values = df_countydata['Max AQI'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colors.values(),
    show_state_data=True,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='Max AQI',
    legend_title='% Air Quality Index'
)

py.iplot(fig, filename='choropleth_full_usa')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~michaelgarza/0 or inside your plot.ly account where it is named 'choropleth_full_usa'


In [56]:
colorscale = ["#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce"]
colors = {"good": "#1fca23", 
          "moderate": "#fffd38", 
          "usg":"#fc6621", 
          "unhealthy": "#fc0d1b", 
          "very unhealthy":"#97084c", 
          "hazardous":"#7d0425", 
          "idk":"#000000"
         }
endpts = [50, 100, 150, 200, 300, 500]
fips = state_aqi['State FIPS Code'].tolist()
values = state_aqi['Median AQI'].tolist()

fig = ff.create_choropleth(
    values=values, scope=['usa'],
    binning_endpoints=endpts,
    colorscale=colors.values(),
    show_state_data=True,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='Median AQI',
    legend_title='% Air Quality Index'
)

py.iplot(fig, filename='choropleth_full_usa')

TypeError: create_choropleth() missing 1 required positional argument: 'fips'

In [50]:
help(ff.create_choropleth)

Help on function create_choropleth in module plotly.figure_factory._county_choropleth:

create_choropleth(fips, values, scope=['usa'], binning_endpoints=None, colorscale=None, order=None, simplify_county=0.02, simplify_state=0.02, asp=None, show_hover=True, show_state_data=True, state_outline=None, county_outline=None, centroid_marker=None, round_legend_values=False, exponent_format=False, legend_title='', **layout_options)
    Returns figure for county choropleth. Uses data from package_data.
    
    :param (list) fips: list of FIPS values which correspond to the con
        catination of state and county ids. An example is '01001'.
    :param (list) values: list of numbers/strings which correspond to the
        fips list. These are the values that will determine how the counties
        are colored.
    :param (list) scope: list of states and/or states abbreviations. Fits
        all states in the camera tightly. Selecting ['usa'] is the equivalent
        of appending all 50 state

In [9]:
# the following block of code is to create the choropleth
# for col in df.columns:
#     df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

colors = {"good": "#1fca23", 
          "moderate": "#fffd38", 
          "usg":"#fc6621", 
          "unhealthy": "#fc0d1b", 
          "very unhealthy":"#97084c", 
          "hazardous":"#7d0425", 
          "idk":"#000000"
         }
endpts = [[50, 100, 150, 200, 300, 500]]

df['text'] = df['state'] + '<br>' +\
    'Beef '+df['beef']+' Dairy '+df['dairy']+'<br>'+\
    'Fruits '+df['total fruits']+' Veggies ' + df['total veggies']+'<br>'+\
    'Wheat '+df['wheat']+' Corn '+df['corn']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['code'],
        z = df['total exports'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout = dict(
        title = '2011 US Agriculture Exports by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )